# Aplicar PCA (Principal Component Analysis)

É um tipo de modelo que utilizamos para gerar uma redução de dimensionalidade nos dados escolhidos sem perder as caracteristicas com um minimo de perda de informação.

